In [1]:
import os
cachedir = '/rscratch/tpang/kinshuk/cache'
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["TRANSFORMERS_CACHE"] = cachedir
os.environ["HF_DATASETS_CACHE"]= cachedir
from model import get_model
from loader.layers import param_count
from loader.data_module import make_data_module
import json
import time
import torch
import random
import logging
import argparse
import numpy as np
import transformers
from pathlib import Path
import torch.backends.mps
import torch.backends.cudnn
from torch.cuda import (
    max_memory_allocated,
    reset_peak_memory_stats,
    reset_max_memory_allocated,
    memory_allocated,
)
from loader.logger import get_logger
from transformers import set_seed
# from accelerate import Accelerator
from torch.utils.data import DataLoader
from os.path import exists, join, isdir
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

from transformers.utils.logging import (
    set_verbosity_error as transformers_vb_err,
)
from datasets.utils.logging import (
    set_verbosity_error as datasets_vb_err,
)
from transformers import Seq2SeqTrainer
from traineval.eval import eval_func
logger = logging.getLogger(__name__)
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
from llamaft import ModelArguments, DataArguments, TrainingArguments, GenerationArguments

/rscratch/tpang/kinshuk/anaconda3/envs/kinbert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-07 13:51:23.784283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 13:51:23.784325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 13:51:23.785724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 13:51:23.794557: I tensorflow/core/platform/cpu_feature_guard.cc:182] Thi

In [2]:
# Setting up the arguments

model_args = ModelArguments(
    model_name_or_path="meta-llama/Llama-2-7b-hf"
)

data_args = DataArguments(
    eval_dataset_size=1024,
    max_eval_samples=1000,
    source_max_len = 1024,
    target_max_len = 256,
    dataset="alpaca", # DATASET [alpaca|chip2|self-instruct|hh-rlhf|oasst1|longform]
)

training_args = TrainingArguments(
    output_dir="./output",
    data_seed=7,
    save_strategy="steps",
    evaluation_strategy="steps",
    logging_strategy="steps",
    do_eval=True,
    eval_steps=187,
    adam_beta2=0.999,

    learning_rate=2e-6,     # LEARNING RATE
    
    max_steps=2000,         # NUMBER OF STEPS

    sortby="alpha",         # CAN DO "alpha" or "lora" or "dora"

    num_layers=12,           # NUMBER OF LAYERS FOR FULL FINE-TUNING

    per_device_train_batch_size = 4, # BATCH-SIZE
    memlog=True,
)

generation_args = GenerationArguments(
    max_new_tokens=256 # default is 256
)

# If you need to use GenerationConfig or similar for generation_args
training_args.generation_config = transformers.GenerationConfig(
    **vars(generation_args)
)

# Combine arguments into a single Namespace object (if needed)
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args),
)
args.cache_dir=cachedir

# Control randomness
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
set_seed(args.seed)  # transformers seed

In [3]:
if 'lora' in args.sortby:
    args.num_layers = 0
logger = logging.getLogger(__name__)
gpus = torch.cuda.device_count()
mempath = (
    f"/rscratch/tpang/kinshuk/RpMKin/llama_ft/{args.dataset}/"
    + f"{args.sortby}"
)
start_memory = [0] * gpus
end_memory = [0] * gpus
peek_memory = 0

if args.verbose:
        task_info = (
            f"\nSeed: {args.seed}\n"
            + f"Dataset: {args.dataset}\n"
            + f"Sort by: {args.sortby}\n"
            + f"Layers to train: {args.num_layers}\n"
        )
        print(task_info)
else:
    datasets_vb_err()
    transformers_vb_err()
    global _tqdm_active
    _tqdm_active = False


Seed: 7
Dataset: alpaca
Sort by: alpha
Layers to train: 12



In [4]:
def memall(gpus=gpus):
    for i in range(gpus):
        start_memory[i] = torch.cuda.memory_allocated(i)
    return sum(start_memory)

model, tokenizer = get_model(args)

for device in range(gpus):
    reset_peak_memory_stats(device=device)
    reset_max_memory_allocated(device=device)

weight_memory = memall()

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.77s/it]


Adding special tokens.
Sorted by  alpha
Training layers: ['model.layers.21.self_attn.v_proj', 'model.layers.22.self_attn.v_proj', 'model.layers.16.self_attn.v_proj', 'model.layers.19.self_attn.v_proj', 'model.layers.18.self_attn.v_proj', 'model.layers.30.self_attn.v_proj', 'model.layers.28.self_attn.v_proj', 'model.layers.17.self_attn.v_proj', 'model.layers.27.self_attn.v_proj', 'model.layers.11.self_attn.v_proj', 'model.layers.23.self_attn.o_proj', 'model.layers.3.mlp.up_proj']
Enabling model.layers.3.mlp.up_proj.weight parameter
Enabling model.layers.11.self_attn.v_proj.weight parameter
Enabling model.layers.16.self_attn.v_proj.weight parameter
Enabling model.layers.17.self_attn.v_proj.weight parameter
Enabling model.layers.18.self_attn.v_proj.weight parameter
Enabling model.layers.19.self_attn.v_proj.weight parameter
Enabling model.layers.21.self_attn.v_proj.weight parameter
Enabling model.layers.22.self_attn.v_proj.weight parameter
Enabling model.layers.23.self_attn.o_proj.weight p

In [5]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

data_module = make_data_module(tokenizer=tokenizer, args=args) # type: ignore
dataset = {k:v for k,v in data_module.items()}
train_dataloader = DataLoader(
    dataset['train_dataset'], # type: ignore
    batch_size=args.per_device_train_batch_size,
    collate_fn=dataset['data_collator'],
    shuffle=True,
)
train_dataloader.__len__()

Splitting train dataset in train and validation according to `eval_dataset_size`


12745

In [6]:
train_losses = []
val_losses = []
val_accs = []
times = []

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
optimizer.zero_grad()
optimizer_memory = 0
forward_time = 0
backward_time = 0

for epoch in range(1):
    train_loss = 0
    tr_steps = 0
    tick = 0
    total_time = 0
    for step, batch in enumerate((train_dataloader)):

        tick = time.time()
        optimizer.zero_grad()
        curr = memall()
        batch = {k: v.to(model.device) for k, v in batch.items()}
        input_memory = memall() - curr
        
        curr = memall()
        start = time.time()
        output = model(**batch)
        forward_time += time.time() - start
        activation_memory = memall() - curr
        
        curr = memall()
        start = time.time()
        # loss = loss_fn(out.logits, batch["labels"]) / args.gradient_accumulation_steps
        loss = output.loss
        loss.backward()
        backward_time += time.time() - start
        gradient_memory = memall() - input_memory - weight_memory - optimizer_memory

        curr = memall()
        optimizer.step()
        if step == 0:
             optimizer_memory = memall() - curr
    
        loss = loss.cpu()
        train_loss += loss.item()
        tr_steps += 1
        train_losses.append(train_loss/tr_steps)
        if step % 50 == 0:
            print(f'Step: {step}, Train Loss: {train_loss/tr_steps}')
        torch.cuda.empty_cache()
        total_time += time.time() - tick
        times.append(total_time)
        if step == args.max_steps:
            model.eval()
            break

total_memory = memall()

Step: 0, Train Loss: 1.649153232574463
Step: 1, Train Loss: 1.6787036657333374
Step: 2, Train Loss: 1.626383900642395
Step: 3, Train Loss: 1.5892391502857208
Step: 4, Train Loss: 1.5332839012145996
Step: 5, Train Loss: 1.5172670483589172
Step: 6, Train Loss: 1.4861642462866647
Step: 7, Train Loss: 1.4683053195476532
Step: 8, Train Loss: 1.4805304606755574
Step: 9, Train Loss: 1.4859204173088074
Step: 10, Train Loss: 1.479223294691606
Step: 11, Train Loss: 1.4619700014591217
Step: 12, Train Loss: 1.4758446491681612
Step: 13, Train Loss: 1.4736823780196053
Step: 14, Train Loss: 1.426924204826355
Step: 15, Train Loss: 1.4374700412154198
Step: 16, Train Loss: 1.4432140799129711
Step: 17, Train Loss: 1.4497534963819716
Step: 18, Train Loss: 1.4462758553655524
Step: 19, Train Loss: 1.4329681813716888
Step: 20, Train Loss: 1.4306757279804774
Step: 21, Train Loss: 1.423410865393552
Step: 22, Train Loss: 1.4256389866704526
Step: 23, Train Loss: 1.4093513488769531
Step: 24, Train Loss: 1.4086719

OutOfMemoryError: CUDA out of memory. Tried to allocate 116.00 MiB. GPU 0 has a total capacty of 47.54 GiB of which 110.94 MiB is free. Including non-PyTorch memory, this process has 46.42 GiB memory in use. Process 2164997 has 1016.00 MiB memory in use. Of the allocated memory 44.65 GiB is allocated by PyTorch, and 1.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
optimizer.zero_grad()
model.eval()
trainer=Seq2SeqTrainer(
                model=model,
                tokenizer=tokenizer,
                args=training_args,
                **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
            )
all_metrics = {"run_name": args.run_name}
if args.do_eval:
    all_metrics = eval_func(args, logger, trainer, all_metrics)
total_memory = memall()
peek_memory = sum([max_memory_allocated(i) for i in range(gpus)])


INFO:__main__:*** Evaluate ***


***** eval metrics *****
  eval_loss               =     1.1208
  eval_runtime            = 0:01:07.76
  eval_samples_per_second =     14.756
  eval_steps_per_second   =      1.845


In [ ]:
all_metrics

{'run_name': './output',
 'eval_loss': 1.120755672454834,
 'eval_runtime': 67.7672,
 'eval_samples_per_second': 14.756,
 'eval_steps_per_second': 1.845}

In [ ]:
memory_string = (
        f"{param_count(model)}\n"
        f"Dataset          : {args.dataset}\n"
        f"Method           : {args.sortby}\n"
        f"Layers           : {args.num_layers}\n"
        f"Batch size       : {args.per_device_train_batch_size}\n"
        f"Learning Rate    : {args.learning_rate}\n"
        f"Forward time     : {forward_time/60} min\n"
        f"Backward time    : {backward_time/60} min\n"
        f"Weight memory    : {weight_memory / 1e6} MB\n"
        f"Optimizer memory : {optimizer_memory / 1e6} MB\n"
        f"Activation memory: {activation_memory / 1e6} MB\n"
        f"Gradient memory  : {gradient_memory / 1e6} MB\n"
        f"Input memory     : {input_memory / 1e6} MB\n"
        f"Total memory     : {total_memory / 1e6} MB\n"
        f"Peak memory      : {peek_memory / 1e6} MB\n"
    )
print(memory_string)

Total params: 6738.42M, Trainable: 265.29M
(6738.423808, 265.293824)
Method           : alpha
Layers           : 8
Learning Rate    : 2e-06
Batch size       : 4
Weight memory    : 27087.929344 MB
Optimizer memory : 2122.350592 MB
Activation memory: 2715.99616 MB
Gradient memory  : 1155.809792 MB
Input memory     : 0.011264 MB
Total memory     : 29304.925696 MB
Peak memory      : 45615.481344 MB



In [ ]:
# SAVE TRAINING HISTORY
base = {"train_loss": train_losses,"time": times}
savepath = f"./output/{args.dataset}/lr_{args.learning_rate}/batch_{args.per_device_train_batch_size}/{args.sortby}/layers_{args.num_layers}"
if args.memlog:
    Path(savepath).mkdir(parents=True, exist_ok=True)
    np.save(os.path.join(savepath, "finetune.npy"), base) # type: ignore
    with open(os.path.join(savepath, "metrics.json"), "w") as fout:
        fout.write(json.dumps(all_metrics))
    log_info = (
        f"\n\n{args.dataset} "
        + f"Batch Size {args.per_device_train_batch_size} "
        + f"{args.sortby} fine-tuning "
        + f"{args.num_layers} Layers"
    )
    logger = get_logger(savepath, "memlog.log")
    logger.info(log_info)
    logger.info(f"\n{memory_string}\n")
    if (args.do_train or args.do_eval or args.do_predict):
        with open(os.path.join(savepath, "metrics.json"), "w") as fout:
            fout.write(json.dumps(all_metrics))